In [1]:
#Neccessary Libraries
import pandas_datareader.data as web
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from scipy.optimize import minimize
from math import e

In [2]:
#Specifies the start and end date for the efficient frontier
start = datetime.datetime(2017,10,1)
end = datetime.datetime(2022,10,1)

In [ ]:
#The stocks that will make up the portfolio
msft = web.DataReader('MSFT','yahoo',start,end)
amt = web.DataReader('AMT','yahoo',start,end)
v = web.DataReader('V','yahoo',start,end)
zts = web.DataReader('ZTS','yahoo',start,end)
txrh = web.DataReader('TXRH','yahoo',start,end)
mkc = web.DataReader('MKC','yahoo',start,end)

In [ ]:
#Concatenates the stock prices
stocks = pd.concat([msft['Close'],amt['Close'],v['Close'],zts['Close'],mkc['Close']], axis = 1)
stocks.columns = ['MSFT','AMT','V','ZTS','MKC']
print(stocks)

In [ ]:
#Produces the Returns
returns = stocks/stocks.shift(1)
#Produces the Log Returns
logReturns = np.log(returns)

In [ ]:
##Gets the Sharpe Ratio 

#This is for number of portfolios
noOfPortfolios = 100000

#In np.zeros the integer is the number of stocks in portfolio
weight = np.zeros((noOfPortfolios,5))
expectedReturn = np.zeros(noOfPortfolios)
expectedVolatility = np.zeros(noOfPortfolios)
sharpeRatio = np.zeros(noOfPortfolios)

meanLogRet = logReturns.mean()
Sigma = logReturns.cov()

for k in range(noOfPortfolios):
    
    # Generate random weight vector
    w = np.array(np.random.random(5))
    w = w/ np.sum(w)
    weight[k,:] = w
    
    # Expected Log Return
    expectedReturn[k] = np.sum(meanLogRet * w)
    
    # Expected Volatility
    expectedVolatility[k] = np.sqrt(np.dot(w.T, np.dot(Sigma,w)))
    
    #Sharpe Ratio
    sharpeRatio[k] = expectedReturn[k]/ expectedVolatility[k]
    

In [ ]:
#Sharpe Ratio Maximization
maxIndex = sharpeRatio.argmax()
weight[maxIndex,:]

In [ ]:
#Minimize the negative sharpe ratio
def negativeSR(w):
    w = np.array(w)
    R = np.sum(meanLogRet * w)
    V = np.sqrt(np.dot(w.T, np.dot(Sigma,w)))
    SR = R/V
    return -1*SR

def checkSumToOne(w):
    return np.sum(w) - 1

#These are the initial weights
w0 = [0.2,0.2,0.2,0.2,0.20]

#The tuple count should be the same as the count of stocks in portfolio
bounds = ((0,1),(0,1),(0,1),(0,1),(0,1))
constraints = ({'type':'eq','fun':checkSumToOne})
w_opt = minimize(negativeSR,w0,method = 'SLSQP',bounds = bounds,constraints = constraints)
w_opt

In [ ]:
#Optimal Weights
w_opt.x

In [ ]:
def minimizeMyVolatility(w):
    w = np.array(w)
    v = np.sqrt(np.dot(w.T, np.dot(Sigma,w)))
    return v

In [ ]:
def getReturn(w):
    w = np.array(w)
    R = np.sum(meanLogRet*w)
    return R

In [ ]:
#Creates the Efficient Frontier Line

#Change the linspace function to the top part of the y-axis
returns = np.linspace(0,0.0013,50)
volatility_opt = []
for R in returns:
    # find best volatility
    constraints = ({'type':'eq','fun':checkSumToOne},
                  {'type':'eq','fun': lambda w: getReturn(w) - R})
    opt = minimize(minimizeMyVolatility,w0,method = 'SLSQP',bounds = bounds,constraints = constraints)
    
    # Save my optimal volatility
    volatility_opt.append(opt['fun'])

In [ ]:
#Scatterplot Returns V. Volatility
plt.figure(figsize = (16,16))
plt.scatter(expectedVolatility,expectedReturn, c = sharpeRatio)
plt.xlabel('Expected Volatility')
plt.ylabel('Expected Log Return')
plt.colorbar(label = 'SR')
plt.scatter(expectedVolatility[maxIndex],expectedReturn[maxIndex],c = 'red')
plt.plot(volatility_opt,returns,'--')
plt.show()

In [ ]:
#This Creates the Covariance Matrix of the Stocks in the portfolio
sn.heatmap(stocks.corr(),annot=True,cmap='BrBG')
plt.title('Stock Correlation Heatmap')
print('')

In [ ]:
#Portfolio Weights Tables with: Volatilities, Log Returns, Returns, and Stocks
portfolioConstruct = pd.DataFrame({'Sharpe_Return':sharpeRatio,
                                  'Log_Volatility':expectedVolatility,
                                  'Log_Return':expectedReturn,
                                   'Sharpe_Ratio':sharpeRatio,
                                  'Return':[e ** i for i in expectedReturn],
                                  'Volatility':[e ** i for i in expectedVolatility]
                                  })
print(portfolioConstruct)

In [ ]:
portfolioConstruct = portfolioConstruct.sort_values(by = ['Sharpe_Return'],ascending = False)
portfolioConstruct['index_column'] = portfolioConstruct.index

In [ ]:
for i in portfolioConstruct.index:
    print(weight[i],portfolioConstruct['Sharpe_Ratio'][i] )
    